# Sloane Lab NHM JSON Parser

This notebook parses the JSON files from the NHM datasets and extracts the data that we want to import into the Sloane Lab knowledge base.

#### Import libraries

In [1]:
# -*- coding: future_fstrings -*-
import os
import re
import sys
import csv
import urllib
import titlecase
import pandas as pd
import xml.etree.ElementTree as ET

import rdflib
from rdflib import Namespace, URIRef, BNode, Literal
from rdflib.namespace import RDF, RDFS, OWL, XSD

#from viapy import api
from bs4 import BeautifulSoup as Soup
from mapboxgl.utils import *
from mapboxgl.viz import *

from matplotlib import *
import matplotlib.pyplot as plt
%matplotlib inline

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.11 from "/opt/homebrew/Cellar/jupyterlab/3.4.8_1/libexec/bin/python3.11"
  * The NumPy version is: "1.24.1"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: No module named 'numpy.core._multiarray_umath'


#### XML File path

In [7]:
# XML file path
DATA_PATH = 'data/nhm/'

#### Load JSON

In [21]:
for dataset_dir in os.listdir(DATA_PATH):

    if not os.path.isfile(dataset_dir):
    
        # Open metadata file
        with open(f'{DATA_PATH}{dataset_dir}/eml.xml') as json_file:
        
            # Read metadata file
            xml_tree = ET.parse('sample.xml')
            xml_root = xml_tree.getroot()
            
            for x in myroot[0]:
                 print(x.tag, x.attrib)

manifest

{'download_id': 'b680f70b-036b-4ae0-b044-6c329d546b42',
 'duration_in_seconds': 4.568718,
 'end': '2022-10-13T13:43:16.039358',
 'file_format': 'csv',
 'files': ['manifest.json', '05ff2255-c38a-40c9-b657-4ccb55ab2feb.csv'],
 'format_args': {},
 'ignore_empty_fields': True,
 'resources': {'05ff2255-c38a-40c9-b657-4ccb55ab2feb': {'field_counts': {'_id': 1307,
    'associatedMedia._id': 1349,
    'associatedMedia.assetID': 1349,
    'associatedMedia.category': 1306,
    'associatedMedia.created': 1349,
    'associatedMedia.creator': 0,
    'associatedMedia.identifier': 1349,
    'associatedMedia.license': 1306,
    'associatedMedia.mime': 1306,
    'associatedMedia.modified': 1349,
    'associatedMedia.rightsHolder': 1306,
    'associatedMedia.title': 1348,
    'associatedMedia.type': 1306,
    'associatedMediaCount': 1306,
    'barcode': 0,
    'basisOfRecord': 1307,
    'bed': 0,
    'catalogNumber': 1307,
    'catalogueDescription': 0,
    'chondriteAchondrite': 0,
    'chronostratigra

#### Declare prefixes and create graph

In [4]:
# Create namespaces
slb=Namespace("http://sloanelab.org/");
ecrm=Namespace("http://erlangen-crm.org/current/");
crmdig=Namespace("http://www.ics.forth.gr/isl/CRMdig/");

# Create graph
graph = rdflib.Graph()

# Bind prefixes
graph.bind("slb", slb)
graph.bind("ecrm", ecrm)
graph.bind("crmdig", crmdig)

#### Add catalogue

In [6]:
for dataset_num, record in enumerate(dataset):
    with open('eggs.csv', newline='') as csvfile:
...     spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    

# Dataset
jsonIRI = slb[f'D1/nhm_dataset{dataset_id}'];
graph.add((jsonIRI, RDF.type, crmdig.D1_Digital_Object))
graph.add((jsonIRI, ecrm.P2_has_type, slb['E55/museum_dataset']))
graph.add((jsonIRI, RDFS.label, Literal('NHM Dataset {dataset_id}')))

for record in dataset['records']:

    # Record
    recordIRI = slb[f'E73/nhm_dataset/{dataset_id}'];
    graph.add((recordIRI, RDF.type, ecrm.E73_Information_Object))
    graph.add((recordIRI, ecrm.P2_has_type, slb['E55/museum_record']))
    graph.add((recordIRI, RDFS.label, Literal('NHM PV P 73123')))

    # People
    for person in record['people']:
        graph.add((recordIRI, ecrm.P67_refers_to, slb[f'E53/{"Malta"}']))
    
    # Places
    for place in record['places']:
        graph.add((recordIRI, ecrm.P67_refers_to, slb[f'E53/{"Malta"}']))
    
    objIRI = slb[f'E19/nhm_dataset/PV_P_73123'];
    graph.add((objIRI, RDF.type, ecrm.E19_Physical_Object))
    graph.add((recordIRI, ecrm.P129_is_about, objIRI))

    graph.add((objIRI, OWL.sameAs, slb[f'E19/fishes/172']))

<Graph identifier=N32ce111042214f02bf6c56079d4b43a8 (<class 'rdflib.graph.Graph'>)>

In [7]:
print(len(graph))

10


In [9]:
graph.serialize(destination='/Users/danielemetilli/Downloads/nhm-shark-tooth.ttl', format="turtle");